# 3D Volume Reconstruction & Analysis Pipeline

This notebook reconstructs 3D segmented volumes from predicted 2D patches and provides comprehensive analysis tools for optic nerve segmentation results.

## Pipeline Overview
1. **Volume Reconstruction** - Assemble 3D volumes from predicted patches
2. **Quality Assessment** - Validate reconstruction accuracy and completeness
3. **Results Analysis** - Generate quantitative and qualitative analysis
4. **Export & Visualization** - Prepare final results for clinical use

## Requirements
- Predicted segmentation masks from inference pipeline
- Original reference images for geometry preservation
- FSL tools for geometry copying (optional)
- Sufficient disk space for 3D volume outputs

In [ ]:
# Core imports
import os
import sys
from pathlib import Path
import logging
from typing import Dict, List, Tuple, Optional
import time
import json

# Scientific computing and visualization
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import nibabel as nib
from tqdm.auto import tqdm

# Statistical analysis
from scipy import ndimage
from sklearn.metrics import jaccard_score

# Import reconstruction functions
from functions import reconstruct_3d_volume

# Configure matplotlib for inline display
%matplotlib inline

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

print("3D Volume Reconstruction & Analysis Pipeline - Initialized")
print(f"Working directory: {os.getcwd()}")
print(f"NumPy version: {np.__version__}")
print(f"NiBabel version: {nib.__version__}")

## 3.1 Configuration & Setup

Configure reconstruction parameters and output directories.

In [ ]:
# =============================================================================
# RECONSTRUCTION CONFIGURATION - Modify these according to your setup
# =============================================================================

# Data paths
DATA_ROOT = Path("data")
PROCESSED_DATA_PATH = DATA_ROOT / "processed"
RAW_DATA_PATH = DATA_ROOT / "raw"

# Input data paths
PREDICTIONS_PATH = PROCESSED_DATA_PATH / "predictions"  # From inference pipeline
REFERENCE_IMAGES_PATH = RAW_DATA_PATH  # Original images for geometry

# Volume parameters
VOLUME_CONFIG = {
    'volume_shape': (176, 240, 165),  # Expected 3D volume dimensions (H, W, D)
    'patch_size': 32,  # Patch size used during extraction
    'image_extension': '.nii.gz'  # File extension for medical images
}

# Processing parameters
INPUT_SIDES = ['left', 'right']
SAMPLE_PATIENTS = []  # Will be auto-detected from prediction data

# Output configuration
OUTPUT_DIRS = {
    'reconstructed': PROCESSED_DATA_PATH / 'reconstructed_volumes',
    'analysis': PROCESSED_DATA_PATH / 'analysis_results',
    'visualizations': PROCESSED_DATA_PATH / 'volume_visualizations',
    'exports': PROCESSED_DATA_PATH / 'final_exports'
}

# Analysis parameters
ANALYSIS_CONFIG = {
    'connectivity_threshold': 0.5,  # Threshold for connected components
    'min_component_size': 10,  # Minimum size for valid components
    'slice_sampling': 5,  # Every nth slice for visualization
    'visualization_slices': 10  # Number of slices to show in overview
}

# Create all output directories
for dir_name, path in OUTPUT_DIRS.items():
    path.mkdir(parents=True, exist_ok=True)
    print(f"  {dir_name}: {path}")

print("\nConfiguration loaded successfully:")
print(f"  Volume shape: {VOLUME_CONFIG['volume_shape']}")
print(f"  Patch size: {VOLUME_CONFIG['patch_size']}")
print(f"  Predictions path: {PREDICTIONS_PATH}")
print(f"  Reference images: {REFERENCE_IMAGES_PATH}")
print(f"  Output directories created: {len(OUTPUT_DIRS)}")

### Data Validation

Validate input predictions and reference images availability.

In [ ]:
def validate_reconstruction_data() -> Dict[str, Dict[str, List[str]]]:
    """Validate prediction data and reference images availability."""
    
    validation_data = {
        'predictions': {},
        'references': {}
    }
    
    # Check predictions directory
    if not PREDICTIONS_PATH.exists():
        raise FileNotFoundError(f"Predictions directory not found: {PREDICTIONS_PATH}")
    
    print("Validating prediction data...")
    total_predictions = 0
    
    for side in INPUT_SIDES:
        side_pred_path = PREDICTIONS_PATH / side
        if not side_pred_path.exists():
            logger.warning(f"Prediction side directory not found: {side_pred_path}")
            validation_data['predictions'][side] = []
            continue
        
        # Find patient directories with predictions
        patients = [d.name for d in side_pred_path.iterdir() if d.is_dir()]
        validation_data['predictions'][side] = patients
        
        print(f"  {side}: {len(patients)} patients with predictions")
        
        for patient in patients[:3]:  # Show details for first 3 patients
            patient_path = side_pred_path / patient
            pred_count = len(list(patient_path.glob('*.nii.gz')))
            total_predictions += pred_count
            print(f"    {patient}: {pred_count} prediction patches")
        
        if len(patients) > 3:
            for patient in patients[3:]:
                patient_path = side_pred_path / patient
                pred_count = len(list(patient_path.glob('*.nii.gz')))
                total_predictions += pred_count
            print(f"    ... and {len(patients) - 3} more patients")
    
    print(f"Total prediction patches: {total_predictions}")
    
    # Check reference images
    print("\nValidating reference images...")
    total_references = 0
    
    if REFERENCE_IMAGES_PATH.exists():
        for side in INPUT_SIDES:
            side_ref_path = REFERENCE_IMAGES_PATH / side
            if side_ref_path.exists():
                ref_files = list(side_ref_path.glob(f'*{VOLUME_CONFIG["image_extension"]}'))
                ref_patients = [f.stem.split('_')[0] for f in ref_files]
                validation_data['references'][side] = ref_patients
                total_references += len(ref_files)
                print(f"  {side}: {len(ref_files)} reference images")
            else:
                validation_data['references'][side] = []
                logger.warning(f"Reference side directory not found: {side_ref_path}")
    else:
        logger.warning(f"Reference images directory not found: {REFERENCE_IMAGES_PATH}")
        for side in INPUT_SIDES:
            validation_data['references'][side] = []
    
    print(f"Total reference images: {total_references}")
    
    # Determine available patients for reconstruction
    all_pred_patients = set()
    for patients in validation_data['predictions'].values():
        all_pred_patients.update(patients)
    
    global SAMPLE_PATIENTS
    SAMPLE_PATIENTS = sorted(list(all_pred_patients))
    
    print(f"\nPatients available for reconstruction: {SAMPLE_PATIENTS}")
    
    return validation_data

# Run validation
validation_data = validate_reconstruction_data()

# Summary
print(f"\nValidation summary:")
print(f"  Patients with predictions: {len(SAMPLE_PATIENTS)}")
print(f"  Sides available: {INPUT_SIDES}")
print(f"  Ready for reconstruction: ✓")

## 3.2 Volume Reconstruction

Reconstruct 3D volumes from predicted segmentation patches.

In [ ]:
def reconstruct_all_volumes() -> Dict[str, str]:
    """Reconstruct 3D volumes for all available patients and sides."""
    
    reconstructed_files = {}
    reconstruction_stats = {
        'successful': 0,
        'failed': 0,
        'total_time': 0
    }
    
    print("Starting 3D volume reconstruction...")
    print(f"Output directory: {OUTPUT_DIRS['reconstructed']}")
    
    start_time = time.time()
    
    for patient_id in tqdm(SAMPLE_PATIENTS, desc="Reconstructing volumes"):
        for side in INPUT_SIDES:
            # Check if predictions exist for this patient-side combination
            pred_path = PREDICTIONS_PATH / side / patient_id
            if not pred_path.exists():
                logger.warning(f"No predictions found for {patient_id}-{side}")
                continue
            
            # Find reference image
            reference_path = None
            ref_side_path = REFERENCE_IMAGES_PATH / side
            
            if ref_side_path.exists():
                # Look for reference image matching patient ID
                ref_pattern = f"{patient_id}_*{VOLUME_CONFIG['image_extension']}"
                ref_files = list(ref_side_path.glob(ref_pattern))
                
                if ref_files:
                    reference_path = str(ref_files[0])
                else:
                    # Try alternative naming patterns
                    alt_patterns = [
                        f"{patient_id}*{side}*{VOLUME_CONFIG['image_extension']}",
                        f"*{patient_id}*{VOLUME_CONFIG['image_extension']}"
                    ]
                    
                    for pattern in alt_patterns:
                        ref_files = list(ref_side_path.glob(pattern))
                        if ref_files:
                            reference_path = str(ref_files[0])
                            break
            
            if not reference_path:
                logger.warning(f"No reference image found for {patient_id}-{side}")
                logger.info(f"Proceeding without geometry copying for {patient_id}-{side}")
                # Create a dummy reference path - the function will handle missing files gracefully
                reference_path = str(ref_side_path / f"{patient_id}_dummy.nii.gz")
            
            try:
                # Perform reconstruction
                output_path = reconstruct_3d_volume(
                    base_patch_dir=str(PREDICTIONS_PATH),
                    patient_id_short=patient_id,
                    side=side,
                    output_base_dir=str(OUTPUT_DIRS['reconstructed']),
                    reference_image_path=reference_path,
                    volume_shape=VOLUME_CONFIG['volume_shape'],
                    patch_size=VOLUME_CONFIG['patch_size']
                )
                
                key = f"{patient_id}-{side}"
                reconstructed_files[key] = output_path
                reconstruction_stats['successful'] += 1
                
                logger.info(f"✓ Reconstructed {key}: {output_path}")
                
            except Exception as e:
                reconstruction_stats['failed'] += 1
                logger.error(f"✗ Reconstruction failed for {patient_id}-{side}: {e}")
                continue
    
    end_time = time.time()
    reconstruction_stats['total_time'] = end_time - start_time
    
    return reconstructed_files, reconstruction_stats

# Perform reconstruction
reconstructed_files, reconstruction_stats = reconstruct_all_volumes()

# Display results
print(f"\nReconstruction completed:")
print(f"  Successful: {reconstruction_stats['successful']}")
print(f"  Failed: {reconstruction_stats['failed']}")
print(f"  Total time: {reconstruction_stats['total_time']:.2f} seconds")
print(f"  Average time per volume: {reconstruction_stats['total_time']/max(1, reconstruction_stats['successful']):.2f} seconds")

print(f"\nReconstructed volumes:")
for key, path in reconstructed_files.items():
    file_size = Path(path).stat().st_size / (1024 * 1024) if Path(path).exists() else 0
    print(f"  {key}: {Path(path).name} ({file_size:.1f} MB)")

## 3.3 Quality Assessment

Analyze reconstruction quality and segmentation characteristics.

In [ ]:
def analyze_volume_quality(volume_path: str) -> Dict:
    """Analyze quality metrics for a reconstructed volume."""
    
    try:
        # Load volume
        volume_img = nib.load(volume_path)
        volume_data = volume_img.get_fdata()
        
        # Basic statistics
        stats = {
            'shape': volume_data.shape,
            'dtype': str(volume_data.dtype),
            'file_size_mb': Path(volume_path).stat().st_size / (1024 * 1024),
            'min_value': float(volume_data.min()),
            'max_value': float(volume_data.max()),
            'mean_value': float(volume_data.mean()),
            'std_value': float(volume_data.std())
        }
        
        # Segmentation-specific metrics
        if volume_data.max() <= 1.0:  # Binary or probability mask
            # Binarize if needed
            binary_mask = (volume_data >= ANALYSIS_CONFIG['connectivity_threshold']).astype(np.uint8)
            
            stats.update({
                'total_voxels': int(volume_data.size),
                'positive_voxels': int(binary_mask.sum()),
                'positive_ratio': float(binary_mask.sum() / volume_data.size),
                'non_zero_slices': int(np.sum(binary_mask.sum(axis=(0, 1)) > 0))
            })
            
            # Connected components analysis
            labeled_array, num_components = ndimage.label(binary_mask)
            
            if num_components > 0:
                component_sizes = [np.sum(labeled_array == i) for i in range(1, num_components + 1)]
                stats.update({
                    'num_components': num_components,
                    'largest_component': max(component_sizes) if component_sizes else 0,
                    'component_sizes': component_sizes[:10]  # Top 10 components
                })
            else:
                stats.update({
                    'num_components': 0,
                    'largest_component': 0,
                    'component_sizes': []
                })
        
        return stats
        
    except Exception as e:
        logger.error(f"Quality analysis failed for {volume_path}: {e}")
        return {'error': str(e)}

# Analyze all reconstructed volumes
def perform_quality_assessment() -> Dict[str, Dict]:
    """Perform quality assessment on all reconstructed volumes."""
    
    quality_results = {}
    
    print("Performing quality assessment...")
    
    for key, volume_path in tqdm(reconstructed_files.items(), desc="Analyzing quality"):
        if not Path(volume_path).exists():
            logger.warning(f"Volume file not found: {volume_path}")
            continue
        
        quality_stats = analyze_volume_quality(volume_path)
        quality_results[key] = quality_stats
    
    return quality_results

# Run quality assessment
quality_results = perform_quality_assessment()

# Display quality summary
print(f"\nQuality Assessment Results:")
print("=" * 50)

for key, stats in quality_results.items():
    if 'error' in stats:
        print(f"\n{key}: ERROR - {stats['error']}")
        continue
    
    print(f"\n{key}:")
    print(f"  Volume shape: {stats['shape']}")
    print(f"  File size: {stats['file_size_mb']:.1f} MB")
    print(f"  Value range: [{stats['min_value']:.3f}, {stats['max_value']:.3f}]")
    
    if 'positive_voxels' in stats:
        print(f"  Segmented voxels: {stats['positive_voxels']:,} ({stats['positive_ratio']:.1%})")
        print(f"  Active slices: {stats['non_zero_slices']}/{stats['shape'][2]}")
        print(f"  Connected components: {stats['num_components']}")
        if stats['largest_component'] > 0:
            print(f"  Largest component: {stats['largest_component']:,} voxels")

# Calculate aggregate statistics
valid_results = [stats for stats in quality_results.values() if 'error' not in stats]

if valid_results:
    avg_file_size = np.mean([stats['file_size_mb'] for stats in valid_results])
    avg_positive_ratio = np.mean([stats.get('positive_ratio', 0) for stats in valid_results])
    
    print(f"\nAggregate Statistics:")
    print(f"  Average file size: {avg_file_size:.1f} MB")
    print(f"  Average segmentation ratio: {avg_positive_ratio:.1%}")
    print(f"  Successfully analyzed: {len(valid_results)}/{len(quality_results)}")

## 3.4 Results Visualization

Generate comprehensive visualizations of reconstructed volumes.

In [ ]:
def visualize_volume_slices(volume_path: str, patient_key: str, 
                           num_slices: int = 6) -> None:
    """Visualize sample slices from a 3D volume."""
    
    try:
        # Load volume
        volume_img = nib.load(volume_path)
        volume_data = volume_img.get_fdata()
        
        # Find slices with content
        slice_sums = volume_data.sum(axis=(0, 1))
        active_slices = np.where(slice_sums > 0)[0]
        
        if len(active_slices) == 0:
            print(f"No active slices found in {patient_key}")
            return
        
        # Select representative slices
        if len(active_slices) >= num_slices:
            slice_indices = np.linspace(0, len(active_slices)-1, num_slices, dtype=int)
            selected_slices = active_slices[slice_indices]
        else:
            selected_slices = active_slices
        
        # Create visualization
        cols = min(3, len(selected_slices))
        rows = (len(selected_slices) + cols - 1) // cols
        
        fig, axes = plt.subplots(rows, cols, figsize=(15, 5*rows))
        if rows == 1 and cols == 1:
            axes = [axes]
        elif rows == 1:
            axes = axes.reshape(1, -1)
        elif cols == 1:
            axes = axes.reshape(-1, 1)
        
        fig.suptitle(f'Volume Reconstruction: {patient_key}', fontsize=16)
        
        for i, slice_idx in enumerate(selected_slices):
            row = i // cols
            col = i % cols
            
            if rows == 1:
                ax = axes[col] if cols > 1 else axes[0]
            else:
                ax = axes[row, col] if cols > 1 else axes[row, 0]
            
            slice_data = volume_data[:, :, slice_idx]
            
            # Display slice
            im = ax.imshow(slice_data, cmap='hot', interpolation='nearest')
            ax.set_title(f'Slice {slice_idx}\nSum: {slice_sums[slice_idx]:.0f}')
            ax.axis('off')
            
            # Add colorbar
            plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
        
        # Hide empty subplots
        for i in range(len(selected_slices), rows * cols):
            row = i // cols
            col = i % cols
            if rows == 1:
                ax = axes[col] if cols > 1 else axes[0]
            else:
                ax = axes[row, col] if cols > 1 else axes[row, 0]
            ax.axis('off')
        
        plt.tight_layout()
        
        # Save visualization
        vis_path = OUTPUT_DIRS['visualizations'] / f'{patient_key}_slices.png'
        plt.savefig(vis_path, dpi=150, bbox_inches='tight')
        print(f"Visualization saved: {vis_path}")
        
        plt.show()
        
    except Exception as e:
        logger.error(f"Visualization failed for {patient_key}: {e}")

# Generate visualizations for all volumes
def generate_all_visualizations(max_volumes: int = 4) -> None:
    """Generate visualizations for reconstructed volumes."""
    
    print(f"Generating visualizations...")
    
    volume_keys = list(reconstructed_files.keys())[:max_volumes]
    
    for key in volume_keys:
        volume_path = reconstructed_files[key]
        if Path(volume_path).exists():
            print(f"\nVisualizing {key}...")
            visualize_volume_slices(volume_path, key, num_slices=6)
        else:
            logger.warning(f"Volume not found for visualization: {volume_path}")

# Generate visualizations
if reconstructed_files:
    generate_all_visualizations(max_volumes=min(4, len(reconstructed_files)))
else:
    print("No reconstructed volumes available for visualization")

### Segmentation Analysis Summary

Comprehensive analysis of segmentation results across all volumes.

In [ ]:
def generate_segmentation_summary() -> Dict:
    """Generate comprehensive segmentation analysis summary."""
    
    summary = {
        'volume_analysis': {},
        'aggregate_metrics': {},
        'quality_flags': {}
    }
    
    valid_volumes = [key for key, stats in quality_results.items() 
                    if 'error' not in stats and 'positive_voxels' in stats]
    
    if not valid_volumes:
        print("No valid volumes found for analysis")
        return summary
    
    print("Generating segmentation analysis summary...")
    
    # Per-volume analysis
    for key in valid_volumes:
        stats = quality_results[key]
        
        volume_summary = {
            'segmentation_volume_mm3': stats['positive_voxels'],  # Assuming 1mm³ voxels
            'segmentation_ratio': stats['positive_ratio'],
            'active_slices': stats['non_zero_slices'],
            'connectivity': stats['num_components'],
            'largest_component_ratio': stats['largest_component'] / max(1, stats['positive_voxels'])
        }
        
        # Quality flags
        flags = []
        if stats['positive_ratio'] < 0.001:  # Less than 0.1% segmented
            flags.append('low_segmentation')
        if stats['positive_ratio'] > 0.1:  # More than 10% segmented (unusual for optic nerve)
            flags.append('high_segmentation')
        if stats['num_components'] > 5:
            flags.append('fragmented')
        if stats['non_zero_slices'] < 5:
            flags.append('sparse_slices')
        
        volume_summary['quality_flags'] = flags
        summary['volume_analysis'][key] = volume_summary
    
    # Aggregate metrics
    all_ratios = [summary['volume_analysis'][key]['segmentation_ratio'] for key in valid_volumes]
    all_volumes = [quality_results[key]['positive_voxels'] for key in valid_volumes]
    all_components = [quality_results[key]['num_components'] for key in valid_volumes]
    all_slices = [quality_results[key]['non_zero_slices'] for key in valid_volumes]
    
    summary['aggregate_metrics'] = {
        'mean_segmentation_ratio': np.mean(all_ratios),
        'std_segmentation_ratio': np.std(all_ratios),
        'mean_volume': np.mean(all_volumes),
        'std_volume': np.std(all_volumes),
        'mean_components': np.mean(all_components),
        'mean_active_slices': np.mean(all_slices)
    }
    
    # Overall quality assessment
    all_flags = [flag for vol in summary['volume_analysis'].values() 
                for flag in vol['quality_flags']]
    flag_counts = {flag: all_flags.count(flag) for flag in set(all_flags)}
    summary['quality_flags'] = flag_counts
    
    return summary

# Generate summary
segmentation_summary = generate_segmentation_summary()

# Display results
print("\nSEGMENTATION ANALYSIS SUMMARY")
print("=" * 50)

if segmentation_summary['aggregate_metrics']:
    metrics = segmentation_summary['aggregate_metrics']
    print(f"\nAggregate Metrics ({len(segmentation_summary['volume_analysis'])} volumes):")
    print(f"  Mean segmentation ratio: {metrics['mean_segmentation_ratio']:.3%} ± {metrics['std_segmentation_ratio']:.3%}")
    print(f"  Mean segmented volume: {metrics['mean_volume']:.0f} ± {metrics['std_volume']:.0f} voxels")
    print(f"  Mean connected components: {metrics['mean_components']:.1f}")
    print(f"  Mean active slices: {metrics['mean_active_slices']:.1f}")

if segmentation_summary['quality_flags']:
    print(f"\nQuality Flags:")
    for flag, count in segmentation_summary['quality_flags'].items():
        print(f"  {flag}: {count} volumes")
else:
    print(f"\n✓ No quality issues detected")

print(f"\nPer-Volume Analysis:")
for key, analysis in segmentation_summary['volume_analysis'].items():
    print(f"\n{key}:")
    print(f"  Segmentation ratio: {analysis['segmentation_ratio']:.3%}")
    print(f"  Volume: {analysis['segmentation_volume_mm3']} voxels")
    print(f"  Active slices: {analysis['active_slices']}")
    print(f"  Components: {analysis['connectivity']}")
    if analysis['quality_flags']:
        print(f"  Quality flags: {', '.join(analysis['quality_flags'])}")
    else:
        print(f"  Quality: ✓ No issues")

## 3.5 Results Export

Export analysis results and prepare final deliverables.

In [ ]:
def export_analysis_results() -> Dict[str, str]:
    """Export all analysis results to structured files."""
    
    export_files = {}
    
    print("Exporting analysis results...")
    
    # 1. Export quality assessment as JSON
    quality_export = {
        'quality_results': quality_results,
        'segmentation_summary': segmentation_summary,
        'reconstruction_stats': reconstruction_stats,
        'configuration': {
            'volume_shape': VOLUME_CONFIG['volume_shape'],
            'patch_size': VOLUME_CONFIG['patch_size'],
            'analysis_config': ANALYSIS_CONFIG
        },
        'timestamp': time.strftime('%Y-%m-%d %H:%M:%S')
    }
    
    quality_file = OUTPUT_DIRS['exports'] / 'quality_assessment.json'
    with open(quality_file, 'w') as f:
        json.dump(quality_export, f, indent=2, default=str)
    export_files['quality_assessment'] = str(quality_file)
    print(f"  Quality assessment: {quality_file}")
    
    # 2. Export file inventory
    file_inventory = {
        'reconstructed_volumes': {key: str(Path(path).name) for key, path in reconstructed_files.items()},
        'volume_paths': reconstructed_files,
        'original_predictions': str(PREDICTIONS_PATH),
        'output_directories': {name: str(path) for name, path in OUTPUT_DIRS.items()}
    }
    
    inventory_file = OUTPUT_DIRS['exports'] / 'file_inventory.json'
    with open(inventory_file, 'w') as f:
        json.dump(file_inventory, f, indent=2)
    export_files['file_inventory'] = str(inventory_file)
    print(f"  File inventory: {inventory_file}")
    
    # 3. Create summary report
    report_content = f"""# 3D Volume Reconstruction Analysis Report

Generated: {time.strftime('%Y-%m-%d %H:%M:%S')}

## Pipeline Summary

### Input Data
- Predictions source: {PREDICTIONS_PATH}
- Reference images: {REFERENCE_IMAGES_PATH}
- Patients processed: {len(SAMPLE_PATIENTS)}
- Sides: {', '.join(INPUT_SIDES)}

### Reconstruction Results
- Successful reconstructions: {reconstruction_stats['successful']}
- Failed reconstructions: {reconstruction_stats['failed']}
- Total processing time: {reconstruction_stats['total_time']:.2f} seconds
- Volume shape: {VOLUME_CONFIG['volume_shape']}
- Patch size: {VOLUME_CONFIG['patch_size']}

### Quality Metrics
"""
    
    if segmentation_summary['aggregate_metrics']:
        metrics = segmentation_summary['aggregate_metrics']
        report_content += f"""
- Mean segmentation ratio: {metrics['mean_segmentation_ratio']:.3%}
- Mean segmented volume: {metrics['mean_volume']:.0f} voxels
- Mean connected components: {metrics['mean_components']:.1f}
- Mean active slices: {metrics['mean_active_slices']:.1f}
"""
    
    report_content += f"""

### Output Files

#### Reconstructed Volumes
"""
    
    for key, path in reconstructed_files.items():
        file_size = Path(path).stat().st_size / (1024 * 1024) if Path(path).exists() else 0
        report_content += f"- {key}: {Path(path).name} ({file_size:.1f} MB)\n"
    
    report_content += f"""

#### Analysis Files
- Quality assessment: quality_assessment.json
- File inventory: file_inventory.json
- Volume visualizations: {OUTPUT_DIRS['visualizations']}

#### Output Directories
"""
    
    for name, path in OUTPUT_DIRS.items():
        report_content += f"- {name}: {path}\n"
    
    report_file = OUTPUT_DIRS['exports'] / 'reconstruction_report.md'
    with open(report_file, 'w') as f:
        f.write(report_content)
    export_files['reconstruction_report'] = str(report_file)
    print(f"  Reconstruction report: {report_file}")
    
    return export_files

# Export results
exported_files = export_analysis_results()

print(f"\nExport completed successfully!")
print(f"All results saved to: {OUTPUT_DIRS['exports']}")

## Pipeline Summary

Complete 3D reconstruction and analysis pipeline summary.

In [ ]:
# Generate final pipeline summary
def generate_final_summary() -> None:
    """Generate comprehensive final summary of the entire pipeline."""
    
    print("=" * 80)
    print("3D VOLUME RECONSTRUCTION & ANALYSIS PIPELINE - FINAL SUMMARY")
    print("=" * 80)
    
    # Input summary
    print(f"\n📊 INPUT DATA SUMMARY")
    print(f"  Patients processed: {len(SAMPLE_PATIENTS)}")
    print(f"  Anatomical sides: {', '.join(INPUT_SIDES)}")
    print(f"  Predictions source: {PREDICTIONS_PATH}")
    print(f"  Reference images: {REFERENCE_IMAGES_PATH}")
    
    # Reconstruction summary
    print(f"\n🔧 RECONSTRUCTION SUMMARY")
    print(f"  Successful reconstructions: {reconstruction_stats['successful']}")
    print(f"  Failed reconstructions: {reconstruction_stats['failed']}")
    print(f"  Success rate: {reconstruction_stats['successful']/(reconstruction_stats['successful']+reconstruction_stats['failed'])*100:.1f}%")
    print(f"  Processing time: {reconstruction_stats['total_time']:.2f} seconds")
    print(f"  Volume dimensions: {VOLUME_CONFIG['volume_shape']}")
    
    # Quality summary
    valid_analyses = len([r for r in quality_results.values() if 'error' not in r])
    print(f"\n🎯 QUALITY ASSESSMENT")
    print(f"  Volumes analyzed: {valid_analyses}/{len(quality_results)}")
    
    if segmentation_summary['aggregate_metrics']:
        metrics = segmentation_summary['aggregate_metrics']
        print(f"  Mean segmentation coverage: {metrics['mean_segmentation_ratio']:.3%}")
        print(f"  Average segmented volume: {metrics['mean_volume']:.0f} voxels")
        print(f"  Average connectivity: {metrics['mean_components']:.1f} components")
    
    # Quality flags summary
    if segmentation_summary['quality_flags']:
        print(f"\n⚠️  QUALITY FLAGS")
        for flag, count in segmentation_summary['quality_flags'].items():
            print(f"  {flag.replace('_', ' ').title()}: {count} volumes")
    else:
        print(f"\n✅ QUALITY STATUS: No issues detected")
    
    # Output summary
    print(f"\n📁 OUTPUT SUMMARY")
    print(f"  Reconstructed volumes: {len(reconstructed_files)}")
    print(f"  Analysis files exported: {len(exported_files)}")
    print(f"  Visualizations generated: {len(list(OUTPUT_DIRS['visualizations'].glob('*.png')))}")
    
    total_size = 0
    for path in reconstructed_files.values():
        if Path(path).exists():
            total_size += Path(path).stat().st_size
    
    print(f"  Total output size: {total_size / (1024**3):.2f} GB")
    
    # File locations
    print(f"\n📂 KEY OUTPUT LOCATIONS")
    for name, path in OUTPUT_DIRS.items():
        file_count = len(list(path.rglob('*'))) if path.exists() else 0
        print(f"  {name.title()}: {path} ({file_count} items)")
    
    # Performance metrics
    if reconstruction_stats['successful'] > 0:
        avg_time = reconstruction_stats['total_time'] / reconstruction_stats['successful']
        print(f"\n⚡ PERFORMANCE METRICS")
        print(f"  Average reconstruction time: {avg_time:.2f} seconds/volume")
        
        if avg_time < 30:
            perf_status = "Excellent"
        elif avg_time < 60:
            perf_status = "Good"
        else:
            perf_status = "Acceptable"
        
        print(f"  Performance rating: {perf_status}")
    
    # Next steps
    print(f"\n🚀 RECOMMENDED NEXT STEPS")
    print(f"  1. Review quality flags and investigate any issues")
    print(f"  2. Validate results against clinical expectations")
    print(f"  3. Use reconstructed volumes for further analysis")
    print(f"  4. Consider parameter optimization if quality is suboptimal")
    
    # Success indicator
    if (reconstruction_stats['successful'] > 0 and 
        valid_analyses == len(quality_results) and
        len(exported_files) > 0):
        status = "🎉 PIPELINE COMPLETED SUCCESSFULLY"
    elif reconstruction_stats['successful'] > 0:
        status = "⚠️ PIPELINE COMPLETED WITH WARNINGS"
    else:
        status = "❌ PIPELINE COMPLETED WITH ERRORS"
    
    print(f"\n{status}")
    print("=" * 80)

# Generate final summary
generate_final_summary()

# Final message
print(f"\n🔬 3D Volume Reconstruction & Analysis Pipeline Complete!")
print(f"📊 All results, visualizations, and analysis reports are ready for review.")
print(f"📁 Check the output directories for detailed results and exported files.")